In [1]:
import csv
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import zipfile
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

In [2]:
#../input/sf-crime/train.csv.zip
z = zipfile.ZipFile('../input/sf-crime/train.csv.zip')
train = pd.read_csv(z.open('train.csv'))
z = zipfile.ZipFile('../input/sf-crime/test.csv.zip')
test = pd.read_csv(z.open('test.csv'))

l = train.Category.unique()
train_cf = ['Category','Resolution','DayOfWeek','PdDistrict','Descript','Address']
test_cf = ['DayOfWeek','PdDistrict','Address']
test_id = test['Id']
cat_train = train['Category']

In [3]:
s1 = list()
for i in train['Dates']:
    dt = datetime.strptime(str(i), '%Y-%m-%d %H:%M:%S')
    s1.append(dt.strftime("%H%M"))
train['Dates']  = pd.DataFrame(s1,columns=['Dates'])

s1 = list()
for i in test['Dates']:
    dt = datetime.strptime(str(i), '%Y-%m-%d %H:%M:%S')
    s1.append(dt.strftime("%H%M"))
test['Dates']  = pd.DataFrame(s1,columns=['Dates'])
    
for i in train_cf:
    train[i] = le.fit_transform(train[i])
    
for i in test_cf:
    test[i] = le.fit_transform(test[i])
train_pred = train['Category']

    
#'2015-05-13 23:53:00'    
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [4]:
test

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2359,3,0,6407,-122.399588,37.735051
1,1,2351,3,0,9744,-122.391523,37.732432
2,2,2350,3,4,6336,-122.426002,37.792212
3,3,2345,3,2,10633,-122.437394,37.721412
4,4,2345,3,2,10633,-122.437394,37.721412
...,...,...,...,...,...,...,...
884257,884257,0001,6,3,7663,-122.408983,37.751987
884258,884258,0001,6,4,5411,-122.425342,37.792681
884259,884259,0001,6,2,11240,-122.445418,37.712075
884260,884260,0001,6,0,4231,-122.387394,37.739479


In [5]:
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2353,37,866,6,4,0,19790,-122.425892,37.774599
1,2353,21,810,6,4,0,19790,-122.425892,37.774599
2,2333,21,810,6,4,0,22697,-122.424363,37.800414
3,2330,16,404,6,4,11,4266,-122.426995,37.800873
4,2330,16,404,6,5,11,1843,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,0015,25,661,1,8,11,15816,-122.459033,37.714056
878045,0001,16,404,1,2,11,11491,-122.447364,37.731948
878046,0001,16,404,1,7,11,11315,-122.403390,37.780266
878047,0001,35,496,1,7,11,22308,-122.390531,37.780607


In [6]:
training = train.drop(['Category','Descript','Resolution'], axis = 1)
testing = test.drop(['Id'], axis = 1)

In [7]:
result = pd.concat([pd.DataFrame(cat_train),pd.DataFrame(train_pred)], axis=1)
result = result.drop_duplicates() 
result.reset_index(drop=True, inplace=True)
#result.columns.name = None
result.columns = ['Id', 'Index']
category = dict(zip(result.Index, result.Id))
result = result.sort_values(by=['Id'])
print(result)

                             Id  Index
25                        ARSON      0
7                       ASSAULT      1
35                   BAD CHECKS      2
28                      BRIBERY      3
9                      BURGLARY      4
24           DISORDERLY CONDUCT      5
21  DRIVING UNDER THE INFLUENCE      6
13                DRUG/NARCOTIC      7
11                  DRUNKENNESS      8
29                 EMBEZZLEMENT      9
33                    EXTORTION     10
26              FAMILY OFFENSES     11
12       FORGERY/COUNTERFEITING     12
18                        FRAUD     13
34                     GAMBLING     14
19                   KIDNAPPING     15
2                 LARCENY/THEFT     16
27                  LIQUOR LAWS     17
31                    LOITERING     18
17               MISSING PERSON     19
5                  NON-CRIMINAL     20
1                OTHER OFFENSES     21
38      PORNOGRAPHY/OBSCENE MAT     22
23                 PROSTITUTION     23
37            RECOVERED V

In [8]:
#reg = LinearRegression()
#reg.fit(training,train_pred)
#model = LogisticRegression(solver='lbfgs')
#model.fit(training,train_pred)
#clf = RandomForestClassifier(n_estimators = 1000, random_state = 9)
#clf.fit(training,train_pred).TPUClusterResolver()
model = KNeighborsClassifier(n_neighbors=len(l))
model.fit(training,train_pred)

KNeighborsClassifier(n_neighbors=39)

In [9]:
predicted= model.predict(testing)

In [10]:
p = predicted.tolist()

In [11]:
output = pd.DataFrame(p,columns=['Output'])
output = output['Output'].map(category)
output

0                  DRUG/NARCOTIC
1                 OTHER OFFENSES
2                  LARCENY/THEFT
3                        ASSAULT
4                        ASSAULT
                   ...          
884257             LARCENY/THEFT
884258             LARCENY/THEFT
884259    FORGERY/COUNTERFEITING
884260                 VANDALISM
884261             LARCENY/THEFT
Name: Output, Length: 884262, dtype: object

In [12]:
output = output.to_dict()


In [13]:
len(output)

884262

In [14]:
df = pd.DataFrame(columns=result['Id'])
zero_mat = np.zeros((len(output), len(result)), dtype=int)
final = pd.DataFrame(data= zero_mat, columns=result['Id'])    
for i in range(0, len(output)):
    c = final.columns.get_loc(output[i])
    final.iloc[i,c] = 1

In [15]:
final.index.name = 'ID'
final.to_csv('./Submission.csv', index = True)